Copyright **`(c)`** 2022 Giovanni Squillero `<squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  


# Lab 3: Policy Search

## Task

Write agents able to play [*Nim*](https://en.wikipedia.org/wiki/Nim), with an arbitrary number of rows and an upper bound $k$ on the number of objects that can be removed in a turn (a.k.a., *subtraction game*).

The player **taking the last object wins**.

* Task3.1: An agent using fixed rules based on *nim-sum* (i.e., an *expert system*)
* Task3.2: An agent using evolved rules
* Task3.3: An agent using minmax
* Task3.4: An agent using reinforcement learning

## Instructions

* Create the directory `lab3` inside the course repo
* Put a `README.md` and your solution (all the files, code and auxiliary data if needed)

## Notes

* Working in group is not only allowed, but recommended (see: [Ubuntu](https://en.wikipedia.org/wiki/Ubuntu_philosophy) and [Cooperative Learning](https://files.eric.ed.gov/fulltext/EJ1096789.pdf)). Collaborations must be explicitly declared in the `README.md`.
* [Yanking](https://www.emacswiki.org/emacs/KillingAndYanking) from the internet is allowed, but sources must be explicitly declared in the `README.md`.

## Deadlines ([AoE](https://en.wikipedia.org/wiki/Anywhere_on_Earth))

* Sunday, December 4th for Task3.1 and Task3.2
* Sunday, December 11th for Task3.3 and Task3.4
* Sunday, December 18th for all reviews

In [ ]:
import logging
from collections import namedtuple
import random
from typing import Callable
from copy import deepcopy
from itertools import accumulate
from operator import xor

## The *Nim* and *Nimply* classes

In [ ]:
Nimply = namedtuple("Nimply", "row, num_objects")

In [ ]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    @property
    def k(self) -> int:
        return self._k

    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects

## Sample (and silly) startegies

In [ ]:
def pure_random(state: Nim) -> Nimply:
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = random.randint(1, state.rows[row])
    return Nimply(row, num_objects)

In [ ]:
def gabriele(state: Nim) -> Nimply:
    """Pick always the maximum possible number of the lowest row"""
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
    return Nimply(*max(possible_moves, key=lambda m: (-m[0], m[1])))

In [ ]:
def nim_sum(state: Nim) -> int:
    *_, result = accumulate(state.rows, xor)
    return result


def cook_status(state: Nim) -> dict:
    cooked = dict()
    cooked["possible_moves"] = [
        (r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1) if state.k is None or o <= state.k
    ]
    cooked["active_rows_number"] = sum(o > 0 for o in state.rows)
    cooked["shortest_row"] = min((x for x in enumerate(state.rows) if x[1] > 0), key=lambda y: y[1])[0]
    cooked["longest_row"] = max((x for x in enumerate(state.rows)), key=lambda y: y[1])[0]
    cooked["nim_sum"] = nim_sum(state)

    brute_force = list()
    for m in cooked["possible_moves"]:
        tmp = deepcopy(state)
        tmp.nimming(m)
        brute_force.append((m, nim_sum(tmp)))
    cooked["brute_force"] = brute_force

    return cooked

In [ ]:
def optimal_startegy(state: Nim) -> Nimply:
    data = cook_status(state)
    return next((bf for bf in data["brute_force"] if bf[1] == 0), random.choice(data["brute_force"]))[0]

In [ ]:
def make_strategy(genome: dict) -> Callable:
    def evolvable(state: Nim) -> Nimply:
        data = cook_status(state)

        if random.random() < genome["p"]:
            ply = Nimply(data["shortest_row"], random.randint(1, state.rows[data["shortest_row"]]))
        else:
            ply = Nimply(data["longest_row"], random.randint(1, state.rows[data["longest_row"]]))

        return ply

    return evolvable

In [ ]:
NUM_MATCHES = 10
NIM_SIZE = 10


def evaluate(strategy: Callable) -> float:
    opponent = (strategy, optimal_startegy)
    won = 0

    for m in range(NUM_MATCHES):
        nim = Nim(NIM_SIZE)
        player = 0
        while nim:
            ply = opponent[player](nim)
            nim.nimming(ply)
            player = 1 - player
        if player == 1:
            won += 1
    return won / NUM_MATCHES

In [ ]:
evaluate(make_strategy({"p": 0.9999}))

0.0

## Oversimplified match

In [ ]:
logging.getLogger().setLevel(logging.DEBUG)

strategy = (make_strategy({"p": 0.1}), optimal_startegy)

nim = Nim(11)
logging.debug(f"status: Initial board  -> {nim}")
player = 0
while nim:
    ply = strategy[player](nim)
    nim.nimming(ply)
    logging.debug(f"status: After player {player} -> {nim}")
    player = 1 - player
winner = 1 - player
logging.info(f"status: Player {winner} won!")

DEBUG:root:status: Initial board  -> <1 3 5 7 9 11 13 15 17 19 21>
DEBUG:root:status: After player 0 -> <1 3 5 7 9 11 13 15 17 19 2>
DEBUG:root:status: After player 1 -> <1 3 5 7 2 11 13 15 17 19 2>
DEBUG:root:status: After player 0 -> <1 3 5 7 2 11 13 15 17 18 2>
DEBUG:root:status: After player 1 -> <1 3 5 7 2 1 13 15 17 18 2>
DEBUG:root:status: After player 0 -> <1 3 5 7 2 1 13 15 17 2 2>
DEBUG:root:status: After player 1 -> <1 3 5 7 2 1 13 15 1 2 2>
DEBUG:root:status: After player 0 -> <0 3 5 7 2 1 13 15 1 2 2>
DEBUG:root:status: After player 1 -> <0 2 5 7 2 1 13 15 1 2 2>
DEBUG:root:status: After player 0 -> <0 2 5 7 2 1 13 0 1 2 2>
DEBUG:root:status: After player 1 -> <0 2 5 7 2 1 2 0 1 2 2>
DEBUG:root:status: After player 0 -> <0 2 5 6 2 1 2 0 1 2 2>
DEBUG:root:status: After player 1 -> <0 2 4 6 2 1 2 0 1 2 2>
DEBUG:root:status: After player 0 -> <0 2 4 5 2 1 2 0 1 2 2>
DEBUG:root:status: After player 1 -> <0 1 4 5 2 1 2 0 1 2 2>
DEBUG:root:status: After player 0 -> <0 0 4 5 2 1 

In [4]:
import random

# Constants
POPULATION_SIZE = 100
MUTATION_RATE = 0.2
MAX_GENERATIONS = 100
num_rows=3
number_of_games = 50

In [ ]:
def nimGame(num_rows):
    nim = [i * 2 + 1 for i in range(num_rows)]
    return nim

def generate_individual(nim):
    """Generate a random integer within the specified bounds."""
    population=[[random.randint(1, nim[i]) for i in range(num_rows)] for _ in range(POPULATION_SIZE)]
    return population

def calculate_fitness(population, nim):
    """Calculate the fitness of an individual based on the absolute difference from the target."""
    fitness_scores = []
    for agent in population:
        score = 0
        for _ in range(number_of_games):
            #nim.reset()
            moves = 0
            while nim:
                ply = agent.make_move(nim)
                nim.nimming(ply)
                moves += 1
                if not nim:
                    score += 2
            score -= moves
        fitness_scores.append(score)
    return fitness_scores

def crossover(parent1, parent2):
    """Perform crossover between two parents to create a new child."""
    a = len(parent1) // 2;
    child = []
    if random.random() < 0.5:
        child.extend(parent1[0:a])
        child.extend(parent2[a:])
    else:
        child.extend(parent2[0:a])
        child.extend(parent1[a:])
    return child


def mutate(genome):
    """Mutate an individual by adding a random value within a range."""
    index=random.randrange(0,len(genome))
    a = random.choice([-1, 1])
    b = random.random()
    c = random.random()
    if (genome[index] == 0 and b <= 0.5):
        genome[index] += 1
    elif (genome[index] == 2 * index + 1 and c <= 0.5):
        genome[index] -= 1
    else:
        genome[index] += a;
    return genome;

def genetic_algorithm():

    nim=nimGame(num_rows)

    population = generate_individual(nim)

    generations = 0
    while generations < MAX_GENERATIONS:
        # Calculate fitness for each individual
        fitness_scores = calculate_fitness(population, nim)

        # Select parents based on fitness
        parents = [population[i] for i in range(POPULATION_SIZE) if random.uniform(0, 1) < fitness_scores[i]]

        # Create the next generation
        next_generation = []

        # Transfer the best individual to the next generation
        best_index = fitness_scores.index(max(fitness_scores))
        next_generation.append(population[best_index])

        # Perform crossover on 70% of the population
        crossover_count = int(0.7 * POPULATION_SIZE)
        for _ in range(crossover_count):
            parent1, parent2 = random.choices(parents, k=2)
            child = crossover(parent1, parent2)
            next_generation.append(child)

        # Perform mutation on 20% of the population
        mutation_count = int(0.2 * POPULATION_SIZE)
        for _ in range(mutation_count):
            parent = random.choice(parents)
            child = mutate(parent, MUTATION_RATE)
            next_generation.append(child)

        # Fill the remaining population with random individuals
        while len(next_generation) < POPULATION_SIZE:
            next_generation.append(generate_individual())

        # Replace the old population with the new generation
        population = next_generation

        generations += 1

        # Print the best individual in each generation
        best_individual = population[fitness_scores.index(max(fitness_scores))]
        print("Generation {}: Best Individual: {}, Fitness: {}".format(generations, best_individual, max(fitness_scores)))

    else:
        print("Maximum generations reached. Target not achieved.")

if __name__ == "__main__":
    genetic_algorithm()

In [7]:
def back_track(self, state):
    moves = [
        (row, min(state.rows[row], max(1, int(self.genome[row] * state.rows[row]))))
        for row in range(len(state.rows)) if state.rows[row] > 0
    ]

    chosen_move = max(moves, key=lambda x: self.genome[x[0]])
    return Nimply(*chosen_move)

In [6]:
def make_move(state):
        xor_sum = 0
        for count in state.rows:
            xor_sum ^= count

        if xor_sum == 0:
            return back_track(state)
        else:
            for row, count in enumerate(state.rows):
                if count > 0:
                    target_count = xor_sum ^ count
                    if target_count < count:
                        return Nimply(row, count - target_count)
            return back_track(state)